In [35]:
import cv2 as cv
import numpy as np

# Tọa độ hình chữ nhật
rect_start = None
rect_end = None
drawing = False

# Tạo ảnh nền trắng
image = np.ones((1920, 1080, 3), dtype=np.uint8) * 255

# Các thông số để transformation
rect_coords = None
trans_x = 0
trans_y = 0
rotation_angle = 0
scale_x = 1.0
scale_y = 1.0
apply_transforms = False

In [36]:
# Hàm vẽ hình chữ nhật
def draw_rectangle(event, x, y, flags, param):
    global rect_start, rect_end, drawing, image, rect_coords, apply_transforms
    
    if event == cv.EVENT_LBUTTONDOWN:
        drawing = True
        rect_start = (x, y)
    
    elif event == cv.EVENT_MOUSEMOVE:
        if drawing:
            img_copy = image.copy()
            cv.rectangle(img_copy, rect_start, (x, y), (0, 255, 0), 2)
            cv.imshow("Draw Rectangle", img_copy)
    
    elif event == cv.EVENT_LBUTTONUP:
        drawing = False
        rect_end = (x, y)
        cv.rectangle(image, rect_start, rect_end, (0, 255, 0), 2)
        rect_coords = (rect_start, rect_end)
        apply_transforms = True  # True thì bắt đầu biến đổi được
        cv.imshow("Draw Rectangle", image)

In [37]:
# Hàm update khi translate
def update_translation(_):
    global trans_x, trans_y, apply_transforms
    trans_x = cv.getTrackbarPos('Trans X', 'Draw Rectangle')
    trans_y = cv.getTrackbarPos('Trans Y', 'Draw Rectangle')
    apply_transforms = True

# Hàm update khi rotate
def update_rotation(_):
    global rotation_angle, apply_transforms
    rotation_angle = cv.getTrackbarPos('Rotation', 'Draw Rectangle')
    apply_transforms = True

# Hàm update khi scale
def update_scale(_):
    global scale_x, scale_y, apply_transforms
    scale_x = cv.getTrackbarPos('Scale X', 'Draw Rectangle') / 100.0
    scale_y = cv.getTrackbarPos('Scale Y', 'Draw Rectangle') / 100.0
    apply_transforms = True

In [38]:
# Hàm chuyển đổi các vị trí để cập nhật hình chữ nhật
import numpy as np
import cv2 as cv

def apply_transformations():
    global image, rect_coords, trans_x, trans_y, rotation_angle, scale_x, scale_y, apply_transforms
    
    if rect_coords is not None and apply_transforms:
        p1, p2 = rect_coords
        img_copy = np.ones_like(image) * 255

        # Translation
        new_p1 = (p1[0] + trans_x, p1[1] + trans_y)
        new_p2 = (p2[0] + trans_x, p2[1] + trans_y)

        # Tọa độ 4 góc của hình chữ nhật đã được dịch
        rect_points = np.array([[new_p1[0], new_p1[1]],
                                [new_p2[0], new_p1[1]],
                                [new_p2[0], new_p2[1]],
                                [new_p1[0], new_p2[1]]], dtype=np.float32)

        # Scalation bằng cách dùng tâm hình chữ nhật
        center = np.mean(rect_points, axis=0)
        scaled_points = []
        for p in rect_points:
            scaled_p = center + (p - center) * np.array([scale_x, scale_y])
            scaled_points.append(scaled_p)

        scaled_points = np.array(scaled_points, dtype=np.float32)

        # Ma trận xoay
        rotation_angle_rad = np.radians(rotation_angle)  # Chuyển đổi góc sang radian
        theta = np.array([[np.cos(rotation_angle_rad), -np.sin(rotation_angle_rad)],
                          [np.sin(rotation_angle_rad), np.cos(rotation_angle_rad)]])

        # Xoay các điểm hình chữ nhật quanh tâm
        rotated_points = []
        for p in scaled_points:
            rotated_p = theta @ (p - center) + center
            rotated_points.append(rotated_p)

        rotated_points = np.array(rotated_points, dtype=np.int32)

        # Vẽ hình chữ nhật đã được xoay
        cv.polylines(img_copy, [rotated_points], isClosed=True, color=(0, 255, 0), thickness=2)

        # Hiển thị hình chữ nhật đã được dịch, scale và xoay
        cv.imshow("Draw Rectangle", img_copy)

        apply_transforms = False

In [39]:
# Tạo window và trackbar để nhận thông tin
cv.namedWindow("Draw Rectangle")
cv.setMouseCallback("Draw Rectangle", draw_rectangle)

# Các trackbar cập nhật thông tin
cv.createTrackbar('Trans X', 'Draw Rectangle', 0, 200, update_translation)
cv.createTrackbar('Trans Y', 'Draw Rectangle', 0, 200, update_translation)
cv.createTrackbar('Rotation', 'Draw Rectangle', 0, 360, update_rotation)
cv.createTrackbar('Scale X', 'Draw Rectangle', 100, 200, update_scale)  # Default is 1.0 (100%)
cv.createTrackbar('Scale Y', 'Draw Rectangle', 100, 200, update_scale)

while True:
    if apply_transforms:
        apply_transformations()
    
    key = cv.waitKey(1) & 0xFF
    if key == 27: 
        break

cv.destroyAllWindows()

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:868: error: (-215:Assertion failed) trackbar in function 'cv::getTrackbarPos'


KeyboardInterrupt: 